In [1]:
from ast import literal_eval
import pandas as pd
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
res = pd.read_csv('data/match_results.csv')

In [3]:
for col in ['w1_or', 'w2_or', 'w3_or', 'w4_or', 
            'w1_nr', 'w2_nr', 'w3_nr', 'w4_nr', 
            'l1_or', 'l2_or', 'l3_or', 'l4_or', 
            'l1_nr', 'l2_nr', 'l3_nr', 'l4_nr', 
           ]:
    res[col].fillna(0, inplace=True)
    res[col] = res[col].astype(int)
    

/var/folders/kz/y62g2z5d6158lcpcksh1fdc40000gp/T/ipykernel_12469/16332807.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  res[col].fillna(0, inplace=True)
/var/folders/kz/y62g2z5d6158lcpcksh1fdc40000gp/T/ipykernel_12469/16332807.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [4]:
start = pd.read_csv('data/match_starts.csv')

In [5]:
start = start.drop_duplicates(subset=['match_id'])

In [6]:
combined = res.merge(start, on='match_id', how='inner')

In [7]:
def w_players(row):
    if row['team_a_won']:
        players = literal_eval(row['team_a_players'])
    else:
        players = literal_eval(row['team_b_players'])
    players = [str(i) for i in players]
    players = players + [''] * (4 - len(players))
    players = '|'.join(players)
    return players


In [8]:
def l_players(row):
    if row['team_a_won']:
        players = literal_eval(row['team_b_players'])
    else:
        players = literal_eval(row['team_a_players'])
    players = [str(i) for i in players]
    players = players + [''] * (4 - len(players))
    players = '|'.join(players)
    return players


In [9]:
combined['winners'] = combined.apply(w_players, axis=1)
combined['losers'] = combined.apply(l_players, axis=1)

In [10]:
combined.head()

,match_id,w1,w2,w3,w4,l1,l2,l3,l4,w1_or,...,l3_nr,l4_nr,team_a_won,team_b_won,date,map,team_a_players,team_b_players,winners,losers
0,1061564,SaraSaiyan,Shadeslayer II,FraGStaR,billuboss,MrWrong,TRush,NewWithPikes,adirath,1458,...,652,583,True,False,2024-11-06,Golden Lakes (Random Civs),"[699611550423711766, 509251341105889280, 56870...","[470492953983582219, 463259002596360203, 50375...",699611550423711766|509251341105889280|56870599...,470492953983582219|463259002596360203|50375940...
1,1061535,SaraSaiyan,TRush,NewWithPikes,adirath,MrWrong,Ju$TK!Dd!nG,FraGStaR,XenrisBlitz,1435,...,501,457,False,True,2024-11-06,Hideout(Random Civs),"[470492953983582219, 714793302582362162, 56870...","[699611550423711766, 463259002596360203, 50375...",699611550423711766|463259002596360203|50375940...,470492953983582219|714793302582362162|56870599...
2,1061516,SaraSaiyan,Shadeslayer II,TRush,XenrisBlitz,KhalDrogo,Ju$TK!Dd!nG,NewWithPikes,adirath,1417,...,646,577,True,False,2024-11-06,Rage Forest (Random Civs),"[699611550423711766, 509251341105889280, 46325...","[811413902176419860, 714793302582362162, 50375...",699611550423711766|509251341105889280|46325900...,811413902176419860|714793302582362162|50375940...
3,1061497,RedViper,Ju$TK!Dd!nG,NewWithPikes,adirath,SaraSaiyan,Shadeslayer II,TRush,XenrisBlitz,1771,...,653,462,False,True,2024-11-06,Golden Pit,"[699611550423711766, 509251341105889280, 46325...","[1286707470802227285, 714793302582362162, 5037...",1286707470802227285|714793302582362162|5037594...,699611550423711766|509251341105889280|46325900...
4,1061471,RedViper,THUNDER,Ju$TK!Dd!nG,adirath,SaraSaiyan,Shadeslayer II,TRush,NewWithPikes,1753,...,673,644,False,True,2024-11-06,Team Acropolis (Random Civs),"[699611550423711766, 509251341105889280, 46325...","[1286707470802227285, 723567999784583239, 7147...",1286707470802227285|723567999784583239|7147933...,699611550423711766|509251341105889280|46325900...


In [11]:
from collections import defaultdict

In [12]:
player_map = defaultdict(list)

In [13]:
def map_players(row):
    winners = [int('0' + i) for i in row['winners'].split('|')]
    losers = [int('0' + i) for i in row['losers'].split('|')]
    w1, w2, w3, w4 = row['w1'], row['w2'], row['w3'], row['w4']
    l1, l2, l3, l4 = row['l1'], row['l2'], row['l3'], row['l4']
    for a, b in zip(winners, [w1, w2, w3, w4]):
        player_map[a].append(b)
    
    for a, b in zip(losers, [l1, l2, l3, l4]):
        player_map[a].append(b)


In [14]:
combined.apply(map_players, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
9442    None
9443    None
9444    None
9445    None
9446    None
Length: 9447, dtype: object

In [15]:
player_map_final = {}
for k, v in player_map.items():
    player_map[k] = Counter(v).most_common()[0][0]

In [16]:
len(player_map)

190

In [17]:
def id_to_name(ids):
    x = []
    for i in ids.split('|'):
        try:
            x.append(player_map[int(i)])
        except:
            x.append('')
    return '|'.join(x)
combined['winners'] = combined['winners'].apply(id_to_name)
combined['losers'] = combined['losers'].apply(id_to_name)

In [18]:
combined.drop(['w1', 'w2', 'w3', 'w4', 'l1', 'l2', 'l3', 'l4', 
               'team_a_won', 'team_b_won', 
               'team_a_players', 'team_b_players'], 
              axis=1, inplace=True)

In [19]:
combined.head(43).tail()

,match_id,w1_or,w2_or,w3_or,w4_or,l1_or,l2_or,l3_or,l4_or,w1_nr,...,w3_nr,w4_nr,l1_nr,l2_nr,l3_nr,l4_nr,date,map,winners,losers
38,1058362,3455,1648,1320,548,2792,1678,1606,920,3476,...,1341,569,2771,1657,1585,899,2024-11-02,Golden Pit,DOOMSTER|Plan B|AMMbush|adirath,addu|RedViper|Royal|BraveHeart
39,1058340,3434,1299,1060,899,2813,1699,1627,569,3455,...,1081,920,2792,1678,1606,548,2024-11-02,Land Nomad (Random Civs),DOOMSTER|AMMbush|THUNDER|BraveHeart,addu|RedViper|Royal|adirath
40,1057854,2317,1371,1269,666,3455,1081,590,515,2338,...,1290,687,3434,1060,569,494,2024-11-01,Beduoins (Random Civs),Water|Saracen|Shadeslayer II|TRush,DOOMSTER|THUNDER|adirath|Garam Aadmi
41,1057824,2294,1348,1246,876,3478,1104,689,538,2317,...,1269,899,3455,1081,666,515,2024-11-01,Arena Island (NC Zone) (Random Civs),Water|Saracen|Shadeslayer II|BraveHeart,DOOMSTER|THUNDER|TRush|Garam Aadmi
42,1057804,3448,1074,659,560,2324,1378,1276,906,3478,...,689,590,2294,1348,1246,876,2024-11-01,Ghost Lake,DOOMSTER|THUNDER|TRush|adirath,Water|Saracen|Shadeslayer II|BraveHeart


In [20]:
c = combined.head(43)
c[['w1', 'w2', 'w3', 'w4']] = c['winners'].str.split('|', expand=True)


/var/folders/kz/y62g2z5d6158lcpcksh1fdc40000gp/T/ipykernel_12469/58338434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c[['w1', 'w2', 'w3', 'w4']] = c['winners'].str.split('|', expand=True)
/var/folders/kz/y62g2z5d6158lcpcksh1fdc40000gp/T/ipykernel_12469/58338434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c[['w1', 'w2', 'w3', 'w4']] = c['winners'].str.split('|', expand=True)
/var/folders/kz/y62g2z5d6158lcpcksh1fdc40000gp/T/ipykernel_12469/58338434.py:2: SettingWithCopyWarning: 
A value is tr

In [21]:
combined[['w1', 'w2', 'w3', 'w4']] = combined['winners'].str.split('|', expand=True)
combined[['l1', 'l2', 'l3', 'l4']] = combined['losers'].str.split('|', expand=True)
                                                                  

In [22]:
combined.drop(['winners', 'losers'], axis=1, inplace=True)

In [23]:
combined['year_month'] = combined['date'].apply(lambda x: x[:7])

In [24]:
combined = combined[['match_id', 'date', 'year_month', 'map', 
                     'w1', 'w2', 'w3', 'w4', 
                     'l1', 'l2', 'l3', 'l4', 
                     'w1_or', 'w2_or', 'w3_or', 'w4_or', 
                     'w1_nr', 'w2_nr', 'w3_nr', 'w4_nr', 
                     'l1_or', 'l2_or', 'l3_or', 'l4_or', 
                     'l1_nr', 'l2_nr', 'l3_nr', 'l4_nr'
                    ]]

In [25]:
combined

,match_id,date,year_month,map,w1,w2,w3,w4,l1,l2,...,w3_nr,w4_nr,l1_or,l2_or,l3_or,l4_or,l1_nr,l2_nr,l3_nr,l4_nr
0,1061564,2024-11-06,2024-11,Golden Lakes (Random Civs),Saracen,Shadeslayer II,FraGStaR,billuboss,MrWrong,TRush,...,518,134,1405,694,669,600,1388,677,652,583
1,1061535,2024-11-06,2024-11,Hideout(Random Civs),Saracen,TRush,newisyou,adirath,MrWrong,Ju$TK!Dd!nG,...,669,600,1428,942,524,480,1405,919,501,457
2,1061516,2024-11-06,2024-11,Rage Forest (Random Civs),Saracen,Shadeslayer II,TRush,Garam Aadmi,KhalDrogo,Ju$TK!Dd!nG,...,671,480,1612,960,664,595,1594,942,646,577
3,1061497,2024-11-06,2024-11,Golden Pit,RedViper,Ju$TK!Dd!nG,newisyou,adirath,Saracen,Shadeslayer II,...,664,595,1437,1342,673,482,1417,1322,653,462
4,1061471,2024-11-06,2024-11,Team Acropolis (Random Civs),RedViper,THUNDER,Ju$TK!Dd!nG,adirath,Saracen,Shadeslayer II,...,940,575,1455,1360,691,662,1437,1342,673,644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9442,394835,2022-01-19,2022-01,African Clearing,AgentP97,bosedk1,wArDaDdY,Simba,Thiru,talentbutnofarm,...,1255,941,1787,1655,1194,877,1773,1641,1180,863
9443,394824,2022-01-19,2022-01,Golden Pit,Thiru,Mirror,Livi,BraveHeart,bosedk1,Himansu,...,1191,778,1400,1303,1252,870,1389,1286,1241,859
9444,394816,2022-01-19,2022-01,Arena,SMR,Thiru,BraveHeart,New Player,talentbutnofarm,Himansu,...,767,767,1668,1322,1318,1265,1655,1303,1305,1252
9445,394792,2022-01-19,2022-01,African Clearing,Morty,Thiru,shrp,CrazyGuY,SMR,Afterlife,...,1268,1202,2335,1320,1185,756,2333,1318,1183,754


In [26]:
def fix_map_name(map_):
    random_civs = False
    if 'random civ' in map_.lower():
        random_civs = True
    map_name = map_.split('(')[0].split('-')[0].strip().title()
    if random_civs:
        map_name += ' (Random civs)'
    return map_name
    

In [27]:
def fix_map_only_name(map_):
    return map_.lower().split('(')[0].split('-')[0].strip().title()
    

In [28]:
combined['map'] = combined['map'].apply(fix_map_name)
combined['map_only'] = combined['map'].apply(fix_map_only_name)

In [29]:
combined.to_csv('data/matches.csv', index=False)

In [30]:
combined

,match_id,date,year_month,map,w1,w2,w3,w4,l1,l2,...,w4_nr,l1_or,l2_or,l3_or,l4_or,l1_nr,l2_nr,l3_nr,l4_nr,map_only
0,1061564,2024-11-06,2024-11,Golden Lakes (Random civs),Saracen,Shadeslayer II,FraGStaR,billuboss,MrWrong,TRush,...,134,1405,694,669,600,1388,677,652,583,Golden Lakes
1,1061535,2024-11-06,2024-11,Hideout (Random civs),Saracen,TRush,newisyou,adirath,MrWrong,Ju$TK!Dd!nG,...,600,1428,942,524,480,1405,919,501,457,Hideout
2,1061516,2024-11-06,2024-11,Rage Forest (Random civs),Saracen,Shadeslayer II,TRush,Garam Aadmi,KhalDrogo,Ju$TK!Dd!nG,...,480,1612,960,664,595,1594,942,646,577,Rage Forest
3,1061497,2024-11-06,2024-11,Golden Pit,RedViper,Ju$TK!Dd!nG,newisyou,adirath,Saracen,Shadeslayer II,...,595,1437,1342,673,482,1417,1322,653,462,Golden Pit
4,1061471,2024-11-06,2024-11,Team Acropolis (Random civs),RedViper,THUNDER,Ju$TK!Dd!nG,adirath,Saracen,Shadeslayer II,...,575,1455,1360,691,662,1437,1342,673,644,Team Acropolis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9442,394835,2022-01-19,2022-01,African Clearing,AgentP97,bosedk1,wArDaDdY,Simba,Thiru,talentbutnofarm,...,941,1787,1655,1194,877,1773,1641,1180,863,African Clearing
9443,394824,2022-01-19,2022-01,Golden Pit,Thiru,Mirror,Livi,BraveHeart,bosedk1,Himansu,...,778,1400,1303,1252,870,1389,1286,1241,859,Golden Pit
9444,394816,2022-01-19,2022-01,Arena,SMR,Thiru,BraveHeart,New Player,talentbutnofarm,Himansu,...,767,1668,1322,1318,1265,1655,1303,1305,1252,Arena
9445,394792,2022-01-19,2022-01,African Clearing,Morty,Thiru,shrp,CrazyGuY,SMR,Afterlife,...,1202,2335,1320,1185,756,2333,1318,1183,754,African Clearing
